In [1]:
with open('H:/College WOrk/papers/stop-words.txt',encoding='utf-16') as fp:
    v = fp.read()

def removestopwords(line):
    #print("original line",line)
    querywords = line.split()

    resultwords  = [word for word in querywords if word.lower() not in v]
    result = ' '.join(resultwords)
    #print(result)
    return result

In [2]:
from nltk.tokenize import word_tokenize
def tokenizefortfidf(originalsen):
    tokenized_sents = [word_tokenize(i) for i in originalsen]
    sen_list1=[]
    for i in tokenized_sents:
        sen_list1=sen_list1+i
    return sen_list1
#print(sen_list1)

In [3]:
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(list(zip(wordlist,wordfreq)))

In [4]:
def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

In [5]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

In [6]:
def computeTF(wordDict, doc):
    tfDict = {}
    corpusCount = len(doc)
    for word, count in wordDict.items():
        tfDict[word] = count/float(corpusCount)
    return(tfDict)

In [7]:
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
   
    for word, val in docList.items():
        idfDict[word] = math.log10(N / float(val) + 1)
        
    return(idfDict)

In [8]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return(tfidf)

In [9]:
lst=[]
def loopsecond(word1len,word1,idf,j):
    for k in range(word1len): 
            
            if idf.index[j] == word1[k]:
                lst.append(idf['ratio'][j])
               
                break
    
def loopfirst(leng,word1,idf):
    word1len = len(word1)
    for j in range(leng):
        #print(j)
        loopsecond(word1len,word1,idf,j)

In [10]:
def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

In [ ]:
from statistics import mean
def Average(lst):
    return mean(lst)

In [ ]:
from rouge.rouge import rouge_n_sentence_level
from rouge import rouge_n_summary_level
from rouge import rouge_l_summary_level
from rouge import rouge_w_sentence_level
from rouge import rouge_w_summary_level
import os
from urllib.parse import urlparse
def comparerouge1(reference,originalfull_str,filename):
    a = urlparse(filename)
    tokenize_words=[]
    tokenize_words1=[]
    for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
            tokenize_words.append(tokenize)
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    with codecs.open(reference+"a2_"+os.path.basename(a.path), encoding='utf-8') as f1:
        data1 = f1.read()
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
    _, _, rouge_1 = rouge_n_summary_level(tokenize_words, tokenize_words1, 1)
    #rouge1.append(rouge_1)
    #print('ROUGE-1: %f' % rouge_1)
    _, _, rouge_2 = rouge_n_summary_level(tokenize_words, tokenize_words1, 2)
    #print('ROUGE-2: %f' % rouge_2)
    #rouge2.append(rouge_2)
    _, _, rouge_l = rouge_l_summary_level(tokenize_words, tokenize_words1)
    #print('ROUGE-L: %f' % rouge_l)
    #rougel.append(rouge_l)
    _, _, rouge_w = rouge_w_summary_level(tokenize_words, tokenize_words1)
    #avg=(rouge_1+rouge_2+rouge_l+rouge_w)/4
    tokenize_words.clear()
    tokenize_words1.clear()
    return rouge_1

In [ ]:
def comparerouge2(reference,originalfull_str,filename):
    a = urlparse(filename)
    tokenize_words=[]
    tokenize_words1=[]
    for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
            tokenize_words.append(tokenize)
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    with codecs.open(reference+"a2_"+os.path.basename(a.path), encoding='utf-8') as f1:
        data1 = f1.read()
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
   
    _, _, rouge_2 = rouge_n_summary_level(tokenize_words, tokenize_words1, 2)
   
    tokenize_words.clear()
    tokenize_words1.clear()
    return rouge_2   

In [ ]:
def comparerougel(reference,originalfull_str,filename):
    a = urlparse(filename)
    tokenize_words=[]
    tokenize_words1=[]
    for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
            tokenize_words.append(tokenize)
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    with codecs.open(reference+"a2_"+os.path.basename(a.path), encoding='utf-8') as f1:
        data1 = f1.read()
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
    
    _, _, rouge_l = rouge_l_summary_level(tokenize_words, tokenize_words1)
    
    tokenize_words.clear()
    tokenize_words1.clear()
    return rouge_l    

In [ ]:
def comparerougew(reference,originalfull_str,filename):
    a = urlparse(filename)
    tokenize_words=[]
    tokenize_words1=[]
    for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
            tokenize_words.append(tokenize)
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    with codecs.open(reference+"a2_"+os.path.basename(a.path), encoding='utf-8') as f1:
        data1 = f1.read()
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
    _, _, rouge_w = rouge_w_summary_level(tokenize_words, tokenize_words1)
    
    tokenize_words.clear()
    tokenize_words1.clear()
    return rouge_w

In [39]:
import codecs
import string
import re
from bs4 import BeautifulSoup  
from nltk.tokenize import word_tokenize
from indicnlp.tokenize import sentence_tokenize
from indicnlp.tokenize import indic_tokenize
import pandas as pd
import sklearn as sk
import math
import io


rouge1=[]
rouge2=[]
rougel=[]
rougew=[]
class ShortText:
    def __init__(self, my_id, human_label, summary, short_text):
        self.id = my_id         
        self.human_label = human_label    
        self.summary = summary 
        self.short_text = short_text
    def __str__(self):
        '''
        For printing purposes.
        '''
        return '%d\t%d\t%s\t%s' % (self.id, self.human_label, self.summary, self.short_text)

def load_file(filename,createfile):
    tokenize_words=[]
    originalsen=[]
    #file1 = open(createfile, "w")
    #retrieve the original text 
    with codecs.open(filename, encoding='utf-8') as f:
        data = f.read()
    #use beautifulsoup to get tag attributes and elements
    soup = BeautifulSoup(data)
    docno=soup.find_all('docno')
    
    text=soup.find_all('text')
    #store in a dictionary with ShortText Instances as values
    instances = {}
    my_id = 0
    for n,tit in zip(docno,text):
        #print("hello2")
        tit=tit.get_text()
        tit=str(tit)
        #print("this is text",tit)
        #s = re.sub('[!#?,.:";]', '', tit)
        for tokenize in indic_tokenize.trivial_tokenize(tit):
            tokenize_words.append(tokenize)
        # Convert list of strings to string
        originalfull_str = convert_list_to_string(tokenize_words)
        senlateron=[]
        sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
        for t in sentences:
            senlateron.append(t)
        originalsen=[]
        sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
        for t in sentences:
            sen=removestopwords(t)
            originalsen.append(sen)
            #print(originalsen)
        sen_listfortfidf=tokenizefortfidf(originalsen)
        #print(sen_listfortfidf)
        list3 = [''.join(c for c in s if c not in string.punctuation) for s in sen_listfortfidf]
        list3 = [s for s in list3 if s]
        dictionary = wordListToFreqDict(list3)
        sorteddict = sortFreqDict(dictionary)
        context=listToString(list3)
        wordDictA = dict.fromkeys(list3, 0)
        for word in list3:
            wordDictA[word]+=1
        df2=pd.DataFrame([wordDictA])
        tfFirst = computeTF(wordDictA, context)
        tf = pd.DataFrame([tfFirst])
        idfs = computeIDF(wordDictA)
        idfFirst = computeTFIDF(tfFirst, idfs)
        #idfSecond = computeTFIDF(tfSecond, idfs)
        #putting it in a dataframe
        idf= pd.DataFrame([idfFirst])
        idf = idf.transpose()
        idf.columns=['ratio']
        final_df = pd.DataFrame(columns=['sentence', 'score'])

        for i in range(len(originalsen)):
            originalsen[i]=str(originalsen[i])[1:-1]
            word = listToString(originalsen[i])
   
            word1 = word.split()
    
            lenofdf=len(idf)
            loopfirst(lenofdf,word1,idf)
            total = sum(lst)
            final_df = final_df.append({'sentence': originalsen[i], 'score': total},ignore_index='True')
    
            lst.clear()
        final_df.sort_values(by=['score'], inplace=True, ascending=False)
        #final_df1=final_df.sort_values(by=['score'], inplace=True, ascending=False)
        #print(final_df)
        final_df2=final_df['sentence'].head(round(50*len(final_df.index)/100))
        #print("this is final_df2",final_df2)
        listtoretrieve=list(final_df2.index.values) 
        listtoretrieve.sort()
        #print(listtoretrieve)
#         #final_df.sort_values(by=['score'], inplace=True, ascending=False)
        #from original list retrive
        df2 = pd.DataFrame(senlateron,columns=['sentences'])
        new_list = df2.values.tolist()
        res1=[]

        for x in new_list:
            res1=res1+x
        finaldraft=""
        for i in listtoretrieve:
            finaldraft+=res1[i]
        finaldraft=" ".join(finaldraft.split()) 
        rouge_1=comparerouge1("C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/economics/annotator2/",finaldraft,filename)
        rouge_2=comparerouge2("C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/economics/annotator2/",finaldraft,filename)
        rouge_l=comparerougel("C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/economics/annotator2/",finaldraft,filename)
        rouge_w=comparerougew("C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/economics/annotator2/",finaldraft,filename)
        
        rouge1.append(rouge_1)
        rouge2.append(rouge_2)
        rougel.append(rouge_l)
        rougew.append(rouge_w)
        
        rouge1average=Average(rouge1)
        rouge2average=Average(rouge2)
        rougelaverage=Average(rougel)
        rougewaverage=Average(rougew)
        print("rouge1 average",rouge1average)
        print("rouge2 average",rouge2average)
        print("rougel average",rougelaverage)
        print("rougew average",rougewaverage)
        #print(finaldraft)
        with io.open(createfile, "w", encoding="utf-8") as f:
            f.write(finaldraft)
       

    return instances

In [ ]:
import os

for root, dirs, files in os.walk("H:/gujarati Text summarization dataset/economics/"):   
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'r') as f:
                load_file(os.path.join(root, file),"H:/TFIDFgujarati Text summarization dataset/economics/"+file)
               